# Overview

Run this script to generate ERA5 and SARAH cutouts using atlite. The two weather data sources used by atlite are:

 - ERA5 (wind and solar)
   - ERA5 data can be downloaded through the Copernicus Climate Data Store cdsapi package, and requires 
registration and setup as per https://cds.climate.copernicus.eu/api-how-to
 - SARAH (solar)
   - SARAH data can be downloaded via from EUMETSAT CMSAF using the portal https://wui.cmsaf.eu/safira/action/viewProduktSearch
Surface Incoming Direct Radiation (SID), Surface Incoming Shortwave Radiation (SIS) and Direct Normal Irradiance (DNI) should be downloaded
and extracted into the sarah_dir path

In [9]:
import atlite
import geopandas as gpd

# Prepare ERA5 cutout

In [20]:
cutout_era5 = atlite.Cutout(
    path="../../cutouts/RSA-2010_22-era5.nc",
    #module=["era5"],
    #x=slice(16, 35),
    #y=slice(-35, -21),
    #time=slice("2020","2021"), # e.g. download 2 years of weather data
    chunks={"time": 100},
)

In [21]:
cutout_era5.data = cutout_era5.data.sel(time=slice("2012","2012"))

In [19]:
cutout_era5.data.sel(time="2012-01-01")

<xarray.Dataset>
Dimensions:           (x: 63, y: 47, time: 24)
Coordinates:
  * x                 (x) float64 16.2 16.5 16.8 17.1 ... 33.9 34.2 34.5 34.8
  * y                 (y) float64 -34.8 -34.5 -34.2 -33.9 ... -21.6 -21.3 -21.0
  * time              (time) datetime64[ns] 2012-01-01 ... 2012-01-01T23:00:00
    lon               (x) float64 dask.array<chunksize=(63,), meta=np.ndarray>
    lat               (y) float64 dask.array<chunksize=(47,), meta=np.ndarray>
Data variables: (12/13)
    height            (y, x) float32 dask.array<chunksize=(47, 63), meta=np.ndarray>
    wnd100m           (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    wnd_azimuth       (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    roughness         (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    influx_toa        (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    influx_direct     (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    ...                ...
    albedo            (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    solar_altitude    (time, y, x) float64 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    solar_azimuth     (time, y, x) float64 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    temperature       (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    soil temperature  (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
    runoff            (time, y, x) float32 dask.array<chunksize=(24, 47, 63), meta=np.ndarray>
Attributes:
    module:             era5
    prepared_features:  ['height', 'wind', 'influx', 'temperature', 'runoff']
    chunksize_time:     100
    dx:                 0.3
    dy:                 0.3
    Conventions:        CF-1.6
    history:            2023-07-31 03:45:16 GMT by grib_to_netcdf-2.25.1: /op...

In [11]:
#cutout_era5.grid.to_file("atlite_cutout_grid.shp")

# Prepare SARAH cutout

In [7]:
cutout_sarah = atlite.Cutout(
    path="../../cutouts/RSA-2012-sarah.nc",
    #module=["sarah"],
    sarah_dir="../../cutouts/sarah_v3",
    #x=slice(16, 35),
    #y=slice(-35, -21),
    #time=slice("2012","2012"),
    chunks={"time": 1000}, # temp fix chunking by lat-1 to resolve error on building the cutout
)
#cutout_sarah.prepare(features="influx")

/home/pklein/anaconda3/envs/atlite/lib/python3.12/site-packages/atlite/cutout.py:190: UserWarning: Arguments sarah_dir are ignored, since cutout is already built.
  warn(


In [8]:
# Merge Sarah and ERA5 cutouts if prepared separately
cutout_sarah.data = cutout_sarah.data.interp(y=cutout_era5.grid.y.unique(), x=cutout_era5.grid.x.unique(), method="linear")
cutout_sarah = cutout_sarah.merge(cutout_era5, compat="override")
cutout_sarah.data.to_netcdf("RSA-2012-sarah.nc")


In [38]:
# Merge cutouts for multiple years

cutout_sarah = atlite.Cutout(
    path="../../cutouts/RSA-2012-sarah.nc",
    chunks={"time": 1000},
)

for y in [2018, 2019, 2020, 2021, 2022]:
    print(y)
    cutout_sarah = cutout_sarah.merge(
        atlite.Cutout(
            path=f"../../cutouts/RSA-{y}-sarah.nc",
            chunks={"time": 1000}
        )
    )
    
cutout_sarah.data = cutout_sarah.data.interp(y=cutout_era5.grid.y.unique(), x=cutout_era5.grid.x.unique(), method="linear")
cutout_sarah = cutout_sarah.merge(cutout_era5, compat="override")

cutout_sarah.data.to_netcdf("RSA-2017_22-sarah.nc")


2018
2019
2020
2021
2022
